# ENSF 544 Data Science for Software Engineers
## Phase 1
#### By: Kyle Friedt, Erslan Salman, Kelvin Tran and Avneet Gill

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

def get_addresses(url):
    addresses = []
    postalcodes = []
    
    zomato_html = get_page_html(url)
    parser = BeautifulSoup(zomato_html, 'html.parser')
    address_tag = parser.find_all("div", class_ = "col-m-16 search-result-address grey-text nowrap ln22")
    
    for address in address_tag:
        postal = None
        temp  = address.get_text()
        temp = temp.strip("\n")
        temp = temp.strip()
        
        if re.search('[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$',temp) != None:
            postal = re.findall('[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$',temp)[0]
            temp = temp.replace(postal,"")

        elif re.search('[A-Za-z]\d[A-Za-z]$',temp) != None:
            postal = re.findall('[A-Za-z]\d[A-Za-z]',temp)[0]
            temp = temp.replace(postal,"")
            
        addresses.append(temp)
        postalcodes.append(postal)
 
    return addresses, postalcodes
    
def get_titles(url):
    titles = []

    zomato_html = get_page_html(url)
    parser = BeautifulSoup(zomato_html, 'html.parser')
    
    title_tag = parser.find_all("a", class_ = "result-title" )
    
    for title in title_tag:
        temp  = title.get_text()
        temp = temp.strip("\n")
        temp = temp.strip()
        titles.append(temp)
    
    return titles
    
def get_next_page(parser):
    links_array = []
    
    next_page = parser.find("a", class_="paginator_item next item")
    next_page = "https://www.zomato.com" + str(next_page['href'])
    next_page = next_page.split("=")
    stripped = next_page[0]
    return stripped

def get_number_of_pages(parser):
    next_page = parser.find("div", class_="col-l-4 mtop pagination-number")
    next_page = next_page.find_all('b')
    number = next_page[1].get_text()
    return number 

def get_all_links(url,number):
    links = []
    links.append(url)
    url_ = url + "="
    
    for x in range(2,int(number)+1):
        new_link = url_+str(x)
        links.append(new_link)
        
    return links
    

def get_page_html(url):    
    headers = {"User-Agent":"Mozilla/5.0"}
    source=requests.get(url, headers=headers).text
    return source


def get_resturant_data(parser):
    url = get_next_page(parser)
    number = get_number_of_pages(parser)
    links = get_all_links(url, number)
    titles = []
    addresses = []
    postal_codes = []
    for link in links:
        title = get_titles(link)
        titles = titles + title
        address, postal = get_addresses(link)
        addresses.extend(address)
        postal_codes.extend(postal)
        
        print("loading...")
    
    df = pd.DataFrame(titles, columns = ['name'])
    df['address'] = addresses
    df['postal_code'] = postal_codes
    return df

url = "https://www.zomato.com/edmonton/downtown-restaurants?page=1"
zomato_html = get_page_html(url)
parser = BeautifulSoup(zomato_html, 'html.parser')

get_resturant_data(parser)

loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...
loading...

,name,address,postal_code
0,Tres Carnales Taqueria,"10119 100A Street, Edmonton",None
1,Corso 32,"10345 Jasper Avenue, Edmonton",T5J1Y7
2,Pampa Brazilian Steakhouse,"9929 109 St NW, Edmonton",T5K 1H6
3,Sabor Restaurant,"10220-103 Street, Edmonton",T5J0Y8
4,Sofra,10345 106th Street NW,T5J 0J2
...,...,...,...
2060,Bloom & Bliss Flower and Coffee Shop,"5115 50 St, Evansburg",T0E0T0
2061,Lily's Steak & Pizza,"4803 Queen St, Bruderheim",T0B0S0
2062,H&D Burger Barn,"4620 50 St, Millet",T0C1Z0
2063,Evansburg Snackbar,"4918 50th Ave, Evansburg",T0A3A0


In [21]:
# open the json file
def get_yelp_json():
    path = 'business.json'
    df = pd.read_json(path, lines=True)
    print(df.shape)
    return df    

In [23]:
# initial cleaning get businesses that are open, in Calgary, and have > 2 stars
# also get relevent columns
def initial_yelp_clean(df):
    df = df[df.is_open == 1]
    df = df[df.city == 'Calgary']
    df = df.loc[:, ('name', 'city', 'address', 'postal_code', 'stars', 
                    'review_count', 'categories')]# .copy()
    df = df[df.stars > 2]
    print(df.shape)
    return df

In [16]:
# get rid of businesses with empty categories
# get rid of businesses without postal code (using postal code for merge)
def drop_yelp_nan(df):
    df['categories'].replace('', np.nan, inplace=True)
    df['postal_code'].replace('', np.nan, inplace=True)
    df = df.dropna()
    print(df.shape)
    return df

In [17]:
# Get businesses that only have Restaurant or Food as a category
def get_restaurants(df):
    tags = ['Restaurants', 'Food']
    mask = df.categories.apply(lambda x: any(item for item in tags if item in x))
    df = df[mask]
    print(df.shape)
    return df

In [18]:
# Exclude rows that have category tags that we don't want
def drop_tags(df):
    tags = ['Gas Stations', 'Convenience Stores', 'Grocery', 'Gyms', 
            'Hunting & Fishing Supplies']
    mask = df.categories.apply(lambda x: not any(item for item in tags if item in x))
    df = df[mask]
    print(df.shape)
    return df

In [19]:
# Get rid of businesses that have the name Liqour in it
def drop_liquor(df):
    df = df[~df['name'].str.contains('Liquor')]
    print(df.shape)
    return df

In [24]:
import pandas as pd
import numpy as np
# funciton calls to get json data and generate a clean dataframe

# generate dataframe with necessary columns
yelp_df = get_yelp_json()
yelp_df = initial_yelp_clean(yelp_df)
yelp_df = drop_yelp_nan(yelp_df)

# clean the data to exclude businesses that are not restaurants
yelp_df = get_restaurants(yelp_df)
yelp_df = drop_tags(yelp_df)
yelp_df = drop_liquor(yelp_df)
yelp_df

(209393, 14)
(5720, 7)
(5673, 7)
(2654, 7)
(2479, 7)
(2452, 7)


,name,city,address,postal_code,stars,review_count,categories
322,Singapore Sam's,Calgary,555 11 Avenue SW,T2R 1P6,3.5,37,"Chinese, Restaurants, Buffets"
378,OTTO Gelato,Calgary,400-5 Avenue SW,T2P 0L6,5.0,13,"Ice Cream & Frozen Yogurt, Food, Coffee & Tea"
603,Asian Buffet,Calgary,"9125 Bonaventure Dr SE, Unit 23",T2J 0P5,2.5,36,"Restaurants, Asian Fusion"
627,Ricky's All Day Grill,Calgary,"8180 11 Street SE, Unit 860",T2H 3B5,3.0,11,"Food, Restaurants, Breakfast & Brunch"
763,Mucho Burrito,Calgary,2312 4th Street SW,T2S 1W9,3.5,17,"Restaurants, Mexican"
...,...,...,...,...,...,...,...
208950,Joey's Seafood Restaurants,Calgary,"9252 MacLeod Trail SE, Unit 25",T2J 0P5,4.0,11,"Fish & Chips, Salad, Restaurants, Seafood"
209060,Famoso Pizzeria,Calgary,5 High Street SE,T2Z,4.0,59,"Restaurants, Pizza"
209062,Tim Hortons,Calgary,222-205 5th Avenue SW,T2P 2V7,3.5,6,"Bakeries, Coffee & Tea, Shopping, Shopping Cen..."
209092,Sushi Hibiki,Calgary,"1 Avenue NE, Unit 630",T2E 0B6,4.0,4,"Sushi Bars, Restaurants"
